Après moultes délibérations, voici le code du dernier df utilisé pour le système de recommandation :

In [2]:
import pandas as pd
import ast
import pickle

In [3]:
link1 = "./BD/tmdb_full.csv"
df_tmdb = pd.read_csv(link1, low_memory = False)

In [30]:
link2 = "./BD/df_t_akas_clean.csv"
df_t_akas = pd.read_csv(link2)

In [49]:
link3 = "./BD/df_t_principals_clean.csv"
df_t_principals = pd.read_csv(link3)

In [52]:
link4 = "./BD/df_basic_name_clean.csv"
df_basic_name_clean = pd.read_csv(link4)

In [20]:
link5 = "./BD/df_final.csv"
df_final = pd.read_csv(link5)
df_final['Genres'] = df_final['Genres'].apply(ast.literal_eval)
df_final['Liste acteurs'] = df_final['Liste acteurs'].apply(ast.literal_eval)
df_final['Réalisateurs'] = df_final['Réalisateurs'].apply(ast.literal_eval)

Nettoyage du df_tmdb

In [ ]:
#L'énoncé nous demandant d'afficher l'affiche du film, on commence par supprimer les lignes qui ne renseignent pas le lien de l'affiche (45413/309572)
df_tmdb_clean = df_tmdb.dropna(subset=['poster_path'])

#Il reste 1 ligne qui renvoie à un film pour public adulte, restons courtois et supprimons la!
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['adult'] == False]

#D'après notre étude de marché nous allons nous limiter aux films Français et Anglais, en faisant le filtre au niveau de la langue
#pour garder les films francophones et anglophones plutôt que français et anglais.
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['original_language'] == 'fr') | (df_tmdb_clean['original_language'] == 'en')]
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['spoken_languages'].str.contains('fr')) | (df_tmdb_clean['spoken_languages'].str.contains('en'))]

#La colonne 'release_date' doit être au format datetime pour filtrer les films > 1960 (suppression de 27335 lignes sur 264158 (10%)) et les classer:
df_tmdb_clean["release_date"] = pd.to_datetime(df_tmdb_clean["release_date"])
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean["release_date"].dt.year > 1960]
df_tmdb_clean = df_tmdb_clean.sort_values('release_date')

#On garde les films avec un runtime > 50  et < 200 pour éviter les courts métrages ou les documentaires de 5h:
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['runtime'] > 50) & (df_tmdb_clean['runtime'] < 200)]

#117 films pas encore sortis, on peut les retirer: 
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['status'] == 'Released']

#Filtrage des lignes avec genre non renseigné
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['genres'].str.len() > 2]
 
#Conversion de la colonne 'genre' en liste
df_tmdb_clean['genres'] = df_tmdb_clean['genres'].apply(ast.literal_eval) 

#On enlève les 'TV movies':
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['genres'].apply(lambda x : 'TV Movie' not in x)]

#Suppression des lignes qui n'ont pas de notes ou qui ont 10/10 car non pertinent
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['vote_average'] > 0) & (df_tmdb_clean['vote_average'] < 10)]

#Gros filtre sur le nb de votants pour la note du film, sinon les résultats ne sont pas pertinents:
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['vote_count'] > 40]

#On enlève les overviews manquant pour avoir une table propre:
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['overview'].isna() == False]

#On va maintenant sélectionner et réaranger les colonnes pertinentes:
#INDEX on vire car on en fera un propre
#ADULT non pertinent car on ne les garde pas
#BACKDROP_PATH non pertinent car on peut avoir les posters
#Seulement 22316/236823 avec un BUDGET > 0 : colonne non pertinente pour ML mais on garde pour afficher si besoin
#genres on garde
#Homepage Non pertinent 
#id servira pour utiliser l'API magique
#imdb_id très utile si besoin de certaines APIs
#original_language interessant on garde
#original_title cool pour le site et pour garder un df clair
#overview sympa pour le site
#popularity génial
#poster_path top pour le site
#production_countries trop complexe a utiliser, on vire
#release_date super pour site
#revenue : Seulement 14% > 0 : colonne non pertinente 
#runtime très utile!
#spoken_language compliqué à utiliser mais il a servi pour filtrer
#status inutile car tous pareils après filtre
#tagline quasi tous vides ou mal rensignés, à virer
#title inutile avec original title+ liste titres
#video inutile car booléen
#vote_average super !
#vote_count on garde pour donner du poids à la note
#pruduction companies inutiles


df_tmdb_clean = df_tmdb_clean[['imdb_id', 'original_title', 'release_date', 'runtime', 'genres', 'original_language', 'vote_average', 'vote_count', 'popularity', 'overview', 'poster_path', 'budget', 'id']]

#On ajoute le liste des titres de films possibles de title akas:
df_tmdb_clean = pd.merge(df_tmdb_clean, df_t_akas, how = 'left', left_on = 'imdb_id', right_on = 'titleId')
df_tmdb_clean = df_tmdb_clean.drop('titleId', axis = 1)

#On renomme les colonnes pour faire jÔli
df_tmdb_clean = df_tmdb_clean.rename(columns= {'imdb_id' : 'ID', 'original_title' : 'Titre', 'release_date' : 'Date de sortie', 'runtime' : 'Durée (minutes)', 'genres' : 'Genres', 
                                               'original_language' : 'Langue Originale', 'vote_average' : 'Note moyenne', 'vote_count' : 'Nb de votes', 'popularity' : 'Score de popularité', 
                                               'overview': 'Résumé', 'poster_path' : 'Lien du poster', 'budget' : 'Budget', 'title' : 'Titres possibles', "id" : "ID_tmdb"})

#Une centaine de liste de titres en NaN: On les drop pour avoir un df propre
df_tmdb_clean = df_tmdb_clean.dropna()

#on supprime les doublons dans les listes:
df_tmdb_clean['Titres possibles'] = df_tmdb_clean['Titres possibles'].apply(ast.literal_eval)
df_tmdb_clean['Titres possibles'] = df_tmdb_clean['Titres possibles'].apply(lambda x : list(set(x)))



#On rajoute un colonne 'décénnie' pour le ML:
df_tmdb_clean['Décénnie'] = df_tmdb_clean['Date de sortie'].dt.year.apply(lambda x : (x//10)*10)  

#TOujours reset_index au cas ou:
df_tmdb_clean = df_tmdb_clean.reset_index().drop('index', axis = 1)

Ajout des colonnes acteurs et realisateur pour avoir un bô df final:

In [ ]:
#On filtre les categories du df_t_principals sur des nouveaux df pour un futur merge:
df_t_principals_act = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_t_principals_real = df_t_principals[df_t_principals['category'] == 'director']

#Préparation nom des personnes en fonction de leur ID:
df_basic_name_noms = df_basic_name_clean[['nconst', 'primaryName']]

#incorporation des noms aux différents df_t_principals:
df_liste_act = pd.merge(df_t_principals_act, df_basic_name_noms, how = 'left', on = 'nconst')
df_liste_real = pd.merge(df_t_principals_real, df_basic_name_noms, how = 'left', on = 'nconst')

#Il existe des NaN qui poseront problème par la suite, on peut les dropper car ils représentent à peine 1% du total:
df_liste_act = df_liste_act.dropna()
df_liste_real = df_liste_real.dropna()

#On drop les colonnes maintenant inutiles 'nconst' et 'category' et on groupby sur les tconst pour le merge:
df_liste_act = df_liste_act.drop(['nconst', 'category'], axis = 1).groupby('tconst').agg(list)
df_liste_real = df_liste_real.drop(['nconst', 'category'], axis = 1).groupby('tconst').agg(list)

#Premier merge pour la colonne "Liste Acteurs":
df_final = pd.merge(df_tmdb_clean, df_liste_act, how = 'left', left_on = 'ID', right_on = 'tconst').rename(columns = {'primaryName' : 'Liste acteurs'})

#2e pour l'ajout de la colonne "Réalisateurs":
df_final = pd.merge(df_final, df_liste_real, how = 'left', left_on = 'ID', right_on = 'tconst').rename(columns = {'primaryName' : 'Réalisateurs'})

#Il y a environ 800 lignnes NaN dans les nouvelles colonnes : on drop pour un df propre:
df_final = df_final.dropna()

#On resest_index
df_final = df_final.reset_index().drop('index', axis = 1)

#Petit export csv pour pas refaire le code à chaque fois et garder transformer automatiquement les colonnes de listes en listes
df_final.to_csv("df_final.csv", index = False)

On veut la listes des acteurs, réals, genres et décénnies sans doublons pour le streamlit

In [83]:
#On créé une liste vide:
acteurs = []
reals = []
genres = []

#Fonction pour un futur "apply" qui ajoute chaque liste du df à la liste vide:
def liste_acteurs(x):
    acteurs.extend(x)

def liste_reals(x):
    reals.extend(x)

def liste_genres(x):
    genres.extend(x)   

df_final['Liste acteurs'].apply(liste_acteurs)
df_final['Réalisateurs'].apply(liste_reals)
df_final['Genres'].apply(liste_genres)

#Pour supprimer les doublons on passe par un set:

acteurs = list(set(acteurs))
reals = list(set(reals))
genres = list(set(genres))
decennies = list(df_final['Décénnie'].unique())

In [84]:
with open('mes_listes.pkl', 'wb') as f:
    pickle.dump(acteurs, f)
    pickle.dump(reals, f)
    pickle.dump(genres, f)
    pickle.dump(decennies, f)

In [90]:
df_final

,ID,Titre,Date de sortie,Durée (minutes),Genres,Langue Originale,Note moyenne,Nb de votes,Score de popularité,Résumé,Lien du poster,Budget,ID_tmdb,Titres possibles,Décénnie,Liste acteurs,Réalisateurs
0,tt0054759,Come September,1961-01-01,112,"[Comedy, Romance]",en,6.900,65,10.214,"Robert Talbot, an American millionaire, arrive...",/6gZXuejbB3yfZrZkIs8u8Wfs8s4.jpg,0,22386,"[Möte i September, Приходи в сентябре, 九月になれば,...",1960,"[Rock Hudson, Gina Lollobrigida, Sandra Dee, B...",[Robert Mulligan]
1,tt0055254,One Hundred and One Dalmatians,1961-01-25,82,"[Adventure, Animation, Comedy, Family]",en,7.167,5646,47.549,When a litter of dalmatian puppies are abducte...,/mRY84MJeWKnp9joev82QtslJFvk.jpg,4000000,12230,"[101 dalmatiyalı, De nacht van de kouwe neuzen...",1960,"[Rod Taylor, Betty Lou Gerson, J. Pat O'Malley...","[Clyde Geronimi, Hamilton Luske, Wolfgang Reit..."
2,tt0055184,The Misfits,1961-02-01,124,"[Western, Drama, Action, Romance]",en,6.942,310,13.136,"While filing for a divorce, beautiful ex-strip...",/pjWlNZeoT5ZcVutOAHkFK7Z22Sb.jpg,4000000,11536,"[Skłóceni z życiem, Inadaptabilii, Mustangy, N...",1960,"[Clark Gable, Marilyn Monroe, Montgomery Clift...",[John Huston]
3,tt0054938,Gorgo,1961-02-02,78,"[Science Fiction, Action, Horror]",en,5.404,68,7.892,Greedy sailors capture a giant lizard off the ...,/8Bs4AmPxmlwHHeGz8QAEsivsgtT.jpg,650000,40804,"[Gorgo: Havets uhyre, Kaijû Gorugo, Gorgo auf ...",1960,"[Bill Travers, William Sylvester, Vincent Wint...",[Eugène Lourié]
4,tt0055571,Underworld U.S.A.,1961-02-22,99,"[Crime, Drama, Thriller]",en,6.900,65,4.572,A bitter young man sets out to get back at the...,/xPlQikmoHV1Rt1n0lwVYXkbWR0I.jpg,0,28699,"[Alles auf eine Karte, Ο υπ' αριθ. 1 εχθρός το...",1960,"[Cliff Robertson, Dolores Dorn, Beatrice Kay, ...",[Samuel Fuller]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16986,tt23219684,Mighty Morphin Power Rangers: Once & Always,2023-04-19,59,"[Action, Adventure, Science Fiction, Family]",en,6.783,161,97.924,"After tragedy strikes, an unlikely young hero ...",/vc87upO8vcAGj9OmgH3AIz6ikKB.jpg,0,1068141,"[Power Rangers: Ayer, hoy y siempre, Могучие Р...",2020,"[Walter Jones, Steve Cardenas, Catherine Suthe...",[Charlie Haskell]
16987,tt5635026,Peter Pan & Wendy,2023-04-20,106,"[Family, Fantasy, Action, Adventure]",en,5.895,277,1358.468,"Wendy Darling, a young girl afraid to leave he...",/9NXAlFEE7WDssbXSMgdacsUD58Y.jpg,0,420808,"[Peter Pan e Wendy, Piotruś Pan i Wendy, 小飛俠與溫...",2020,"[Alexander Molony, Ever Anderson, Jude Law, Al...",[David Lowery]
16988,tt20115096,A Tourist's Guide to Love,2023-04-21,96,"[Romance, Comedy, Adventure]",en,6.543,69,161.027,"After an unexpected break up, a travel executi...",/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg,0,813726,"[Útikönyv a szerelemhez, Маршрут до кохання, G...",2020,"[Rachael Leigh Cook, Scott Ly, Ben Feldman, Mi...",[Steven K. Tsuchida]
16989,tt27197387,AKA,2023-04-28,122,"[Thriller, Action, Crime]",fr,7.024,148,988.329,A steely special ops agent finds his morality ...,/3BSxAjiporlwQTWzaHZ9Yrl5C9D.jpg,0,1102776,"[AKA, Также известный, Kimdim Ben?, Alias, 千面特...",2020,"[Alban Lenoir, Éric Cantona, Thibault de Monta...",[Morgan S. Dalibert]
